In [1]:
from quant.quant_backtrack_base import get_trade_days

tradedays = get_trade_days('2021-09-01', '')

print(tradedays)


['2021-09-01', '2021-09-02', '2021-09-03', '2021-09-06', '2021-09-07', '2021-09-08', '2021-09-09', '2021-09-10', '2021-09-13', '2021-09-14', '2021-09-15', '2021-09-16', '2021-09-17', '2021-09-22', '2021-09-23', '2021-09-24', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30', '2021-10-08', '2021-10-11', '2021-10-12', '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-18', '2021-10-19', '2021-10-20', '2021-10-21', '2021-10-22', '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28', '2021-10-29', '2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05', '2021-11-08', '2021-11-09', '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15', '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19', '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-25', '2021-11-26', '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03', '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10', '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17', '2021

In [6]:

from stockbase.stock_reader import get_kdf_from_pkl

df = get_kdf_from_pkl(600036)

dfu = df[df['date'] <= '2021-09-04']




In [ ]:
import time

from stockbase.stock_reader import get_kdf_from_pkl


class Strategy(object):

    def __init__(self):
        pass

    def run(self, ctx, stock, date):
        df = ctx.dfs[stock]
        if df is not None:
            df = df[df['date'] <= date]

            close = df['close'].values[-1]
            open = df['open'].values[-1]
            close2 = df['close'].values[-2]

            if open > close2 and close > open:
                ctx.buy(stock, 100)
                return 1
            else:
                ctx.sell(stock, 100)
                return -1

        pass


class QuantBacktest(object):

    def __init__(self):
        self.dfs = {}
        self.selectfun = None
        self.strategy = Strategy()
        self.orders = []
        self.positions = {}

        self.startdate = '2021-09-01'
        self.enddate = ''

        self.curday = ''
        self._stocks = []
        pass

    def buy(self, s, amount):
        if s in self.positions.keys():
            self.positions[s] = self.positions[s] + amount
            self.orders.append((self.curday, 'buy', s, amount))
        else:
            self.positions[s] = amount
            self.orders.append((self.curday, 'buy', s, amount))
        pass

    def sell(self, s, amount):
        if s in self.positions.keys():
            pa = self.positions[s]
            if pa >= amount:
                self.positions[s] = self.positions[s] - amount
                self.orders.append((self.curday, 'sell', s, amount))
            else:
                self.positions[s] = 0
        else:
            self.orders.append((self.curday, 'sell0', s, amount))
            pass

        pass

    def update_data(self, date):
        for s in self._stocks:
            if s not in self.dfs.keys():
                self.dfs[s] = get_kdf_from_pkl(s)
            else:
                pass
        pass

    def select_stocks(self, date):
        self._stocks = [600036]
        pass

    def trade_stocks(self, date):
        for s in self._stocks:
            self.strategy.run(self, s, date)
        pass

    def run(self):

        t0 = time.time()
        tradedays = get_trade_days(self.startdate, self.enddate)
        print(f'run start {tradedays[0]}')
        for d in tradedays:
            print(d)
            t00 = time.time()
            self.curday = d

            self.select_stocks(d)

            self.update_data(d)

            self.trade_stocks(d)

            et00 = time.time() - t00
            print(f'run {d} {et00:.3f} s')

        et0 = time.time() - t0
        print(f'run finish {tradedays[0]}- {tradedays[-1]} by {et0:.3f} s')
        pass


t = QuantBacktest()

t.startdate = '2021-09-01'

t.run()


